### Determining the coating structure from measured spectral data
The question we are trying to answer here is: given a *measurement* of the power transmissivity T as a function of the laser wavelength $\lambda$, can we determine the layer structure of the dielectric coating that the coating is comprised of? I attempt to answer this in a staged way:
 - First, assume the coating consists of a known number of doublets of some thickness, and known dispersion.
 - Next, can we allow for a custom cap layer?
 - Next, can we vary the number of doublets in the structure, and also recover some information about the dispersion of the dielectrics?
 - Finally, can individual layers have custom thicknesses?

We will use the `emcee` package to help us evaluate the most likely parameter set ${\theta_i}$ for each of these cases. The complexity of the problem essentially boils down to increasing the dimensionality of the set of parameters we want to extract from presumably a single measurement (although it is easy to imagine using more information, e.g. $T(\lambda)$ for two different polarizations.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import emcee, corner
from matplotlib.ticker import FormatStrFormatter
import sys, os
sys.path.append('../generic/')
import coatingUtils as coatUtil
import scipy.io as scio
import timeit
import tqdm
from scipy.interpolate import interp1d, PchipInterpolator
import scipy.stats

In [ ]:
# Directory setup
figDir = 'Figures/'
if figDir.strip('/') not in os.listdir():
    os.mkdir(figDir)
    print('Figure directory didnt exist. Made it just now.')
try:
    plt.style.use(['fivethirtyeight', 'gvELOG'])
except:
    pass
saveFig = False

### Dummy coating
For a start, let's make a dummy coating design. I choose to replicate the repetitive structure specified for the [40m PR3/SR3 mirrors](https://dcc.ligo.org/LIGO-E1800089). Explicitly, this consists of 19 layer pairs of \[0.2805, 0.265\] \[SiO2 Ta2O5\] layers, with a cap of [0.5, 0.2805] (all numbers are *optical* thicknesses, i.e. $\frac{nl}{\lambda}$, where $n$ is the refractive index at wavelength $\lambda$ and $l$ is the layer thickness). The task here is to determine 4 numbers, which are the optical thickness of (i) top layers of SiO2 ($l_1$) and Ta2O5 ($l_2$), and (ii) repetitive layer structure [$l_3$, $l_4$] 19 times.

In [ ]:
# Define the layer structure
layers = np.tile([0.2805, 0.265], 19)
layers = np.hstack((np.array([0.5, 0.2805]), layers))
# Load some dispersion data - since we can only infer optical thickness, we can't yet break this degeneracy in the product nl
dispersion = scio.loadmat('../PRC/dispersion_revised.mat')

# Define the wavelength vector over which to evaluate reflectivity
nPts = 201
wavelengths = np.linspace(0.8, 1.2, nPts)
Rmeas, Tmeas, lmeas = coatUtil.specREFL(layers, '../PRC/dispersion_revised.mat', 
                                        aoi=41.1, lam=wavelengths) # This is our "measurement for this dummy experiment"

In [ ]:
# Since we know the wavelength vector, we can do the interpolation one-time and save some time
f_nSiO2 = PchipInterpolator(dispersion['SiO2'][:,0], dispersion['SiO2'][:,1],
                        extrapolate=True)
f_nTa2O5 = PchipInterpolator(dispersion['Ta2O5'][:,0], dispersion['Ta2O5'][:,1],
                    extrapolate=True)
nSiO2 = f_nSiO2(wavelengths*1064)
nTa2O5 = f_nTa2O5(wavelengths*1064)
nSiO2_0 = f_nSiO2(1064)
nTa2O5_0 = f_nTa2O5(1064)

for kk, ii in enumerate(range(nPts)):
        n_c = np.tile(np.array([nSiO2[kk], nTa2O5[kk]]), 20)
        n_c = np.hstack((np.array([1]), n_c, np.array([nSiO2[kk]])))
        if kk == 0:
            n_cTot = n_c
        elif kk != 0:
            n_cTot = np.vstack((n_cTot, n_c))

# Some correction factor
corFactor = np.copy(n_cTot)
corFactor[:,1::2] /= nSiO2_0
corFactor[:,2::2] /= nTa2O5_0

In [ ]:
def specREFLFaster(n, L, lamb, corFactor, theta=0, pol='te'):
    G1, Z1 = multidiel1(n, L, lamb, corFactor, theta=theta, pol=pol)
    return(1 - np.abs(G1)**2)

def multidiel1(n, L, lamb, corFactor, theta=0, pol='te'):
    '''
    Calculates reflectivity and compelx impedance of a dielectric stack.
    This is a faster version of coatingUtils.multidiel1, with the 
    main speedup due to the fact that it can handle a matrix of 
    refractive indices corresponding to layers (columns) and wavelengths (rows)

    Parameters:
    -----------
    n: numpy.ndarray
        p by q array of refractive indices, including the incident and 
        transmitted media. Ordered from incident medium to 
        transmitted medium. p indexes the wavelengths, and q indexes layers.
    L: array_like
        Array of optical thicknesses comprising the dielectric 
        stack, ordered from incident medium to transmitted medium.
        Should have 2 fewer elements than n.
    lamb: float or array_like
        Wavelength(s) at which the reflectivity is to be evaluated, 
        in units of some central (design) wavelength. 
    theta: float
        Angle of incidence in degrees. 
        Defaults to 0 degrees (normal incidence)
    pol: str, 'te' or 'tm'
        Polarization at which reflectivity is to be evaluated. 
        Defaults to 'te' (s-polarization)

    Returns:
    --------
    Gamma1: complex
        Amplitude reflectivity for the input dielectric stack.
    Z1:
        Complex impedance at the interface 
    Example usage:
    --------------
    r_p, _ = multidiel1(n, L, [1.0, 0.5], 45.3, 'te')
    evaluates the amplitude reflectivity for the dielectric stack 
    specified by n and L (which are wavelength dependent in general), 
    at a design wavelength and the second harmonic wavelength, 
    at an angle of incidence of 45.3 degrees for 'te' polarized (s-pol) light.

    References:
    -----------
    [1]: http://eceweb1.rutgers.edu/~orfanidi/ewa/
    '''
    M = np.shape(n)[1]-2                # number of slabs
    if M == 0:
        L = np.array([])
    theta = theta * np.pi / 180
    costh = np.conj(np.sqrt(np.conj(1 - (n[None,:,0].T * np.sin(theta) / n)**2)))
    if (pol=='te' or pol=='TE'):
        nT = n * costh
    else:
        nT = n / costh
    if M > 0:
        L = L[None] * costh[:, 1:M+1]
        L *= corFactor[:,1:-1]
    r = -np.diff(nT) / (np.diff(nT) + 2*nT[:, 0:M+1])
    Gamma1 = r[:,M] * np.ones_like(lamb)
    for ii in range(M-1,-1,-1):
        delta = 2*np.pi*L[:,ii]/lamb
        z = np.exp(-2*1j*delta)
        Gamma1 = (r[:,ii] + Gamma1*z) / (1 + r[:,ii]*Gamma1*z)
    Z1 = (1 + Gamma1) / (1 - Gamma1)
    return Gamma1,Z1

In [ ]:
# Next, we need to setup some functions to run the emcee sampler
def lnlike(theta, Tmeas, lmeas, n_c, corFactor):
    '''
    Likelihood function given T(lambda) data and a model.
    
    Parameters:
    ------------
    theta: np.ndarray
        A 5 element array, which are optical thicknesses of 1st two, 
        and repeating layers of SiO2/Ta2O5, and a standard deviation respectively.
    Tmeas: np.ndarray
        Measured T(lambda) data
    lmeas: np.ndarray
        wavelength vector as a fraction of some central wavelength.
    n_c: np.ndarray
        A matrix of refractive indices at the wavelengths of interest, for dispersion.
        
    Returns:
    --------
    like: float
        The NLL defined in the standard way.
    '''
    # Some error handling
    if len(Tmeas) != len(lmeas):
        print('Transmission data {} and wavelength vector {} do not have the same shape.'.format(np.shape(Tmeas), np.shape(lmeas)))
        return
    elif len(theta) != 5:
        print('Parameter vector has length {}, expected a length 5 vector'.format(len(theta)))
        return
    else:
        l1, l2, l3, l4, sigma = theta
        layers = np.tile([l3, l4], 19)
        layers = np.hstack((np.array([l1, l2]), layers))
        #_, Tmodel, _ = coatUtil.specREFL(layers, '../PRC/dispersion_revised.mat', aoi=41.1, lam=lmeas)
        Tmodel = specREFLFaster(n_c, layers, wavelengths, corFactor, theta=41.1, pol='tm')
        like = -0.5* np.sum(np.log(2*np.pi*sigma**2) + (Tmodel-Tmeas)**2/sigma**2)
        return(like)
    
def lnprior(theta, truths, priorType='uniform'):
    '''
    Prior function, assuming product of i.i.d. Gaussian distributions or uniform distributions.
    
    Parameters:
    ------------
    theta: np.ndarray
        A 5 element array, which are optical thicknesses of 1st two, 
        and repeating layers of SiO2/Ta2O5, and a standard deviation respectively.
    truths: np.ndarray
        A 5 element array of the true values to assume for theta.
    priorType: str
        Type of prior distribution to calculate. 'gaussian' or 'uniform'.
    '''
    std = truths[-1]
    prod = 1/truths[-1]
    if theta[-1] < 0:
        prod = 0      # Guarantee positive standard deviation
        return(np.log(prod))
            
    if priorType != 'uniform' and priorType != 'gaussian':
        print('{} is not a recognized type of prior distribution. Use \'uniform\' or \'gaussian\'.'.format(priorType))
        return
    elif priorType == 'uniform':
        if np.any(theta[:-1] < (1-std)*truths[:-1]) or np.any(theta[:-1] > (1+std)*truths[:-1]):
            prod = 0
        else:
            for jj in truths[:-1]:
                prod *= 1/(std*jj)
    elif priorType == 'gaussian':
        for kk, jj in zip(theta[:-1], truths[:-1]):
            prod *= (1./((std*jj)*np.sqrt(2*np.pi))) * np.exp(-(kk-jj)**2/(2*std*jj)**2)
    return(np.log(prod))

def lnpost(theta, truths, Tmeas, lmeas, n_c, corFactor, priorType='gaussian'):
    '''
    Posterior distribution.
    '''
    return(lnprior(theta, truths, priorType) + lnlike(theta, Tmeas, lmeas, n_c, corFactor))

In [ ]:
theta0 = np.array([0.5, 0.25, 0.25, 0.25, 0.03])
truths = np.array([0.5, 0.2805, 0.2805, 0.265, 0.01])
nDim = len(theta0)
nWalkers = 100
nBurn = 1000
nSteps = 6000 # Seems like we need this many samples to have the autocorrelation time warning not be displayed
starting_guesses = np.random.normal(theta0, 0.01*np.ones(len(theta0))*theta0, size=(nWalkers, nDim))

In [ ]:
# This cell takes ~200 seconds to run for a 400 point wavelength grid
sampler = emcee.EnsembleSampler(nWalkers, nDim, lnpost, args=[truths, Tmeas, wavelengths, n_cTot, corFactor])
pos, prob, state = sampler.run_mcmc(starting_guesses, nBurn, progress=True)
sampler.reset()
tic = timeit.default_timer()
sampler.run_mcmc(pos, nSteps, progress=True)
toc = timeit.default_timer()

In [ ]:
print("Mean acceptance fraction: {0:.3f}".format(np.mean(sampler.acceptance_fraction)))
print('Sampling time was {} s'.format(round(toc-tic,3)))

In [ ]:
fig, ax = plt.subplots(nDim, nDim, figsize=(24,24))
fig.subplots_adjust(hspace=0.2,wspace=0.2)
for aa in ax:
    for bb in aa:
        bb.xaxis.labelpad = 40
        bb.yaxis.labelpad = 40
corner.corner(sampler.flatchain, truths=truths,
             labels=['$l_1$ [$\\lambda$]', '$l_2$ [$\\lambda$]', '$l_{\\mathrm{SiO_2}}$ [$\\lambda$]', '$l_{\\mathrm{Ta_2O_5}}$ [$\\lambda$]', '$\\sigma$'],
             range=[0.99, 0.99, 0.99, 0.99, 0.99],
             show_titles=True, title_fmt='.4f', density=True, fig=fig)

In [ ]:
# What does the transmission curve look like?
l1, l2, l3, l4, _ = np.median(sampler.flatchain, axis=0)
l_MCMC = np.tile(np.array([l3, l4]), 19)
l_MCMC = np.hstack((np.array([l1,l2]), l_MCMC))
T_MCMC = specREFLFaster(n_cTot, l_MCMC, wavelengths, corFactor, theta=41.1, pol='tm')
fig2, ax2 = plt.subplots(1, 1, sharex=True, figsize=(16,9))
ax2.semilogy(wavelengths*1064, Tmeas, label='Measurement')
ax2.semilogy(wavelengths*1064, T_MCMC, label='From emcee', linestyle='--')
ax2.xaxis.set_major_formatter(FormatStrFormatter("%2d"))
ax2.legend(loc='best')
ax2.set_xlabel('Wavelength, $\lambda$ [nm]')
ax2.set_ylabel('Power transmissivity, T')
fig2.suptitle('Inferring coating structure from T($\lambda$) data');

In [ ]:
sampler.acor